# Lab 12-1 many to one
### word sentiment classification 
* many to one 
* variable input sequence length

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
%matplotlib inline

print(tf.__version__)

1.12.0


### Prepairing dataset

In [2]:
# example data
words = ['good', 'bad', 'worse', 'so good']
y_data = [1,0,0,1]

# generating a Token dictionary
char_set = ['<pad>'] + sorted(list(set(''.join(words))))
idx2char = {idx : char for idx, char in enumerate(char_set)}
char2idx = {char : idx for idx, char in enumerate(char_set)}

print(char_set)
print(idx2char)
print(char2idx)

['<pad>', ' ', 'a', 'b', 'd', 'e', 'g', 'o', 'r', 's', 'w']
{0: '<pad>', 1: ' ', 2: 'a', 3: 'b', 4: 'd', 5: 'e', 6: 'g', 7: 'o', 8: 'r', 9: 's', 10: 'w'}
{'<pad>': 0, ' ': 1, 'a': 2, 'b': 3, 'd': 4, 'e': 5, 'g': 6, 'o': 7, 'r': 8, 's': 9, 'w': 10}


In [3]:
# converting Token to Idx
x_data = list(map(lambda word : [char2idx.get(char) for char in word], words))
x_data_len = list(map(lambda word : len(word), x_data))

print(x_data)
print(x_data_len)

[[6, 7, 7, 4], [3, 2, 4], [10, 7, 8, 9, 5], [9, 7, 1, 6, 7, 7, 4]]
[4, 3, 5, 7]


In [4]:
# padding 
max_sequence = 10
x_data = pad_sequences(sequences = x_data, maxlen = max_sequence, padding='post',
                       truncating='post')

print(x_data)
print(x_data_len)
print(y_data)

[[ 6  7  7  4  0  0  0  0  0  0]
 [ 3  2  4  0  0  0  0  0  0  0]
 [10  7  8  9  5  0  0  0  0  0]
 [ 9  7  1  6  7  7  4  0  0  0]]
[4, 3, 5, 7]
[1, 0, 0, 1]


In [5]:
# generating data pipeline 
tr_dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
tr_dataset = tr_dataset.batch(batch_size = 2).repeat()
print(tr_dataset)

<RepeatDataset shapes: ((?, 10), (?,)), types: (tf.int32, tf.int32)>


### Creating model

In [6]:
num_classes = 2
hidden_dim = 10

input_dim = len(char2idx)
output_dim = len(char2idx)
one_hot = np.eye(len(char2idx))

model = Sequential()
model.add(layers.Embedding(input_dim=input_dim, output_dim=output_dim,
                                 trainable=False, mask_zero=True,input_length=max_sequence,
                                 embeddings_initializer=keras.initializers.Constant(one_hot)))
model.add(layers.SimpleRNN(units=hidden_dim))
model.add(layers.Dense(units=num_classes, activation='softmax'))

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 11)            121       
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 10)                220       
_________________________________________________________________
dense (Dense)                (None, 2)                 22        
Total params: 363
Trainable params: 242
Non-trainable params: 121
_________________________________________________________________


### Training model

In [8]:
lr = .01
opt = tf.train.AdamOptimizer(learning_rate = lr)

model.compile(optimizer=opt,
              loss=keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

In [9]:
model.fit(tr_dataset, epochs=30, steps_per_epoch=4//2)

Epoch 1/30
2/2 [==============================] - 0s 163ms/step - loss: 0.6284 - acc: 0.7500
Epoch 2/30
2/2 [==============================] - 0s 6ms/step - loss: 0.4830 - acc: 1.0000
Epoch 3/30
2/2 [==============================] - 0s 10ms/step - loss: 0.3766 - acc: 1.0000
Epoch 4/30
2/2 [==============================] - 0s 10ms/step - loss: 0.2925 - acc: 1.0000
Epoch 5/30
2/2 [==============================] - 0s 8ms/step - loss: 0.2264 - acc: 1.0000
Epoch 6/30
2/2 [==============================] - 0s 7ms/step - loss: 0.1748 - acc: 1.0000
Epoch 7/30
2/2 [==============================] - 0s 8ms/step - loss: 0.1345 - acc: 1.0000
Epoch 8/30
2/2 [==============================] - 0s 10ms/step - loss: 0.1032 - acc: 1.0000
Epoch 9/30
2/2 [==============================] - 0s 9ms/step - loss: 0.0792 - acc: 1.0000
Epoch 10/30
2/2 [==============================] - 0s 11ms/step - loss: 0.0609 - acc: 1.0000
Epoch 11/30
2/2 [==============================] - 0s 8ms/step - loss: 0.0472 - acc

### Checking performance

In [10]:
yhat = model.predict(x_data)
yhat = np.argmax(yhat, axis=-1)

In [11]:
print('acc : {:.2%}'.format(np.mean(yhat == y_data)))

acc : 100.00%
